# NYC Motor Vehicle Collisions - Data Cleaning
In the following we are going to profile and clean the NYC Motor Vehicle Collisions dataset, which contains collisions occured from 2012 to 2021. The dataset consists of over 1.8 million rows and the compressed data file is about 73 MB.
We will use [`OpenClean`](https://github.com/VIDA-NYU/openclean)  and [`geopy`](https://pypi.org/project/geopy/) to profile and clean the data.

Before we start, let us configure the environment
  
  `pip install openclean`

  `pip install openclean-geo`
  
  `pip install geopy`

  `pip install humanfriendly`
  







# Datasets and Streams
The identifier of the vehicle collisions dataset is `h9gi-nx95`. The following code downloads the dataset in tab-delimited CSV format and it will be stored in a local file called `h9gi-nx95.tsv.gz`.

In [1]:
# Download the full 'Motor Vehicle Collisions - Crashes' dataset.
import gzip
import humanfriendly
import os
from openclean.data.source.socrata import Socrata

dataset = Socrata().dataset('h9gi-nx95')

datafile = './h9gi-nx95.tsv.gz'

# Download file only if it does not exist already.
if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as f:
        print('Downloading ...\n')
        dataset.write(f)

fsize = humanfriendly.format_size(os.stat(datafile).st_size)
print("Using '{}' in file {} of size {}".format(dataset.name, datafile, fsize))

Using 'Motor Vehicle Collisions - Crashes' in file ./h9gi-nx95.tsv.gz of size 73.12 MB


In [2]:
# Open the downloaded dataset to extract the relevant columns and records.
from openclean.pipeline import stream
import pandas as pd
import numpy as np

datafile = './h9gi-nx95.tsv.gz'
ds = stream(datafile)
df_full = ds.to_df()

According to the entropy (we will calculate later) of each attribute, some data have little impact on the results, like CONTRIBUTING FACTOR VEHICLE 3-5 and VEHICLE TYPE CODE 3-5. So we will remove them.

In [3]:
# select the subset of columns

df = df_full[['CRASH DATE',
            'CRASH TIME',
            'BOROUGH',
            'ZIP CODE',
            'LATITUDE',
            'LONGITUDE',
            'LOCATION',
            'ON STREET NAME', 
            'CROSS STREET NAME',
            'OFF STREET NAME',
            'NUMBER OF PERSONS INJURED',
            'NUMBER OF PERSONS KILLED',
            'NUMBER OF PEDESTRIANS INJURED',
            'NUMBER OF PEDESTRIANS KILLED',
            'NUMBER OF CYCLIST INJURED',
            'NUMBER OF CYCLIST KILLED',
            'NUMBER OF MOTORIST INJURED',
            'NUMBER OF MOTORIST KILLED',
            'CONTRIBUTING FACTOR VEHICLE 1',
            'CONTRIBUTING FACTOR VEHICLE 2',
            'COLLISION_ID',
            'VEHICLE TYPE CODE 1',
            'VEHICLE TYPE CODE 2']]

# Data Profiling
We use the default column profiler from `openclean` to compute basic statistics as the number of empty values, distinct values, etc.

In [4]:
# Profile the resulting dataset view using the default data profiler
from openclean.profiling.dataset import dataset_profile

profile = dataset_profile(df)

In [5]:
# Print overview of profiling results
profile.stats()

,total,empty,distinct,uniqueness,entropy
CRASH DATE,1841953,0,3426,0.001860,11.679380
CRASH TIME,1841953,0,1440,0.000782,8.929098
BOROUGH,1841953,569042,5,0.000004,2.118309
ZIP CODE,1841953,569264,232,0.000182,7.221128
LATITUDE,1841953,215215,122377,0.075228,15.632447
LONGITUDE,1841953,215215,95989,0.059007,15.342376
LOCATION,1841953,215215,243622,0.149761,16.181466
ON STREET NAME,1841953,376607,16022,0.010934,10.587271
CROSS STREET NAME,1841953,662529,19249,0.016321,11.808361
OFF STREET NAME,1841953,1557715,179278,0.630732,16.909494


In [6]:
# Print data types for each column.
profile.types()

,date,float,int,str
CRASH DATE,3426,0,0,0
CRASH TIME,0,0,0,1440
BOROUGH,0,0,0,5
ZIP CODE,0,0,231,1
LATITUDE,0,122376,1,0
LONGITUDE,0,95987,2,0
LOCATION,0,0,0,243622
ON STREET NAME,86,0,16,15920
CROSS STREET NAME,2,1,26,19220
OFF STREET NAME,30,0,1,179247


In [7]:
# Print the minimum and maximum value for column 'CRASH DATE'
profile.minmax('CRASH DATE')

,min,max
date,2012-07-01,2021-11-16


In [8]:
# Print the minimum and maximum value for column 'CRASH TIME'
profile.minmax('CRASH TIME')

,min,max
str,0:00,9:59


# Data Cleaning



## Missing Geographic Information

As for motor collision data, geographic attributes are vital. This is also reflected in the entropy of each attribute. So, if all the vital information is missing, the record is useless, it will be deleted. Specifically, the row with empty BOROUGH, ZIP CODE, STREET NAME(ON/OFF/CROSS) and LOCATION(LATITUDE/ LONGITUDE) will be deleted.


In [9]:
# drop all records of lost geographical attributes
df = df.drop(df[(df['LOCATION'] == '') & \
                (df['ON STREET NAME'] == '') & \
                (df['OFF STREET NAME'] == '') &  \
                (df['CROSS STREET NAME'] == '') & \
                (df['BOROUGH'] == '') & \
                (df['ZIP CODE'] == '')].index)

For all the missing longtitude/latitude/on street name/cross street name/off street name, there will be difficult to calculate the location through Map API to navigate the other values

In [10]:
# drop all records that cannot calculate location 
df = df.drop(df[((df['LONGITUDE'] == '')  | (df['LONGITUDE'] == '0')) &\
                (df['ON STREET NAME'] == '') & (df['OFF STREET NAME'] == '') &  
                (df['CROSS STREET NAME'] == '')].index)

## Fill Geographic Information
For calculating other missing geological values, importing geopy library into the data cleaning process. With specific LONGITUDE and LATITUDE, using librabry to get zipcodes and address is farly easy and tidy

However, the problem of using Geocoding API is it has usage limits which only allows 50 requests per second; Ideally, if we process 50 records per second, it costs approximately 25 days to process the whole dataset which is not time efficient. We did not find a better way to solve this problem, so we only calculate 20 records for demostration.

In [11]:
# Using an open source library geopy to fill empty location
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="data-cleaning-project")

def find_zipcode(location):
    address = geolocator.geocode(location)
    arr = address.raw['display_name'].split(', ')
    return arr[len(arr) - 2]

def find_borough(location):
    address = geolocator.geocode(location)
    arr = address.raw['display_name'].split(', ')
    return arr[len(arr) - 5]

def find_street(location):
    address = geolocator.geocode(location)
    arr = address.raw['display_name'].split(', ')
    street = ''
    for i in range(1, len(arr) - 5):
        street += arr[i] + ' '
    return street

def find_latlng(street):
    location = geolocator.geocode(street)
    try:
        lat = location.latitude
        lng = location.longitude
    except AttributeError:
        lat = 0
        lng = 0
    return (lat, lng)

def find_location(x):
    if x['ON STREET NAME'] != '':
        return find_latlng(x['ON STREET NAME'])
    elif x['CROSS STREET NAME'] != '':
        return find_latlng(x['CROSS STREET NAME'])
    elif x['OFF STREET NAME'] != '':
        return find_latlng(x['OFF STREET NAME'])
    else:
        return x['LOCATION']

In [12]:
# find location, latitude, langitude by street name (only)
df.loc[df['LOCATION'] == '', 'LOCATION'] = df.loc[df['LOCATION'] == ''].head(20).apply(find_location, axis=1)

## Uppercase
Standardizing all the String type value is clear for professionals to analyze in the future

In [13]:
# Change all words to upper case
df.apply(lambda x: x.astype(str).str.upper())

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2
0,04/14/2021,5:32,,,,,"(40.80205005, -73.8297471344276)",BRONX WHITESTONE BRIDGE,,,...,0,0,0,0,0,FOLLOWING TOO CLOSELY,UNSPECIFIED,4407480,SEDAN,SEDAN
1,04/13/2021,21:35,BROOKLYN,11217,40.68358,-73.97617,"(40.68358, -73.97617)",,,620 ATLANTIC AVENUE,...,0,0,0,0,0,UNSPECIFIED,,4407147,SEDAN,
2,04/15/2021,16:15,,,,,"(40.849642349999996, -73.83640393750001)",HUTCHINSON RIVER PARKWAY,,,...,0,0,0,0,0,PAVEMENT SLIPPERY,,4407665,STATION WAGON/SPORT UTILITY VEHICLE,
3,04/13/2021,16:00,BROOKLYN,11222,,,"(42.083058, -76.05075)",VANDERVORT AVENUE,ANTHONY STREET,,...,0,0,0,0,0,FOLLOWING TOO CLOSELY,UNSPECIFIED,4407811,SEDAN,
4,04/12/2021,8:25,,,0,0,"(0.0, 0.0)",EDSON AVENUE,,,...,0,0,0,0,0,UNSPECIFIED,UNSPECIFIED,4406885,STATION WAGON/SPORT UTILITY VEHICLE,SEDAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1841948,07/06/2012,15:09,MANHATTAN,10035,40.8012354,-73.9418153,"(40.8012354, -73.9418153)",EAST 119 STREET,PARK AVENUE,,...,0,0,0,0,0,UNSPECIFIED,UNSPECIFIED,59654,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE
1841949,07/03/2012,17:30,QUEENS,11102,40.7747112,-73.9333863,"(40.7747112, -73.9333863)",27 AVENUE,4 STREET,,...,0,0,0,2,0,FAILURE TO YIELD RIGHT-OF-WAY,UNSPECIFIED,272592,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON
1841950,07/01/2012,15:30,BROOKLYN,11236,40.6450318,-73.9199775,"(40.6450318, -73.9199775)",RALPH AVENUE,CLARENDON ROAD,,...,0,0,0,0,0,UNSPECIFIED,UNSPECIFIED,135041,SMALL COM VEH(4 TIRES),PASSENGER VEHICLE
1841951,07/08/2012,18:30,,,40.7861217,-73.8040782,"(40.7861217, -73.8040782)",,,,...,0,0,0,0,0,UNSPECIFIED,UNSPECIFIED,3055617,PASSENGER VEHICLE,PASSENGER VEHICLE


## Street Format
Normalize the street name to more proper format by using `StandardizeUSStreetName` from `openclean_geo`

In [14]:
# Use street name standardization operator to modify street names
from openclean_geo.address.usstreet import StandardizeUSStreetName
f = StandardizeUSStreetName(characters='upper', alphanum=True, repeated=False)
df['ON STREET NAME'] = f.apply(df['ON STREET NAME'], threads=3)
df['CROSS STREET NAME'] = f.apply(df['CROSS STREET NAME'], threads=3)
df['OFF STREET NAME'] = f.apply(df['OFF STREET NAME'], threads=3)

## Error  Data Type 

There are some rows with error data type，like int and date type in ON STREET NAME, CROSS STREET NAME, OFF STREET NAME. Regular expression is a good way to solve it.

In [15]:
# Replace the error type data like int, date
df['ON STREET NAME'] = df['ON STREET NAME'].str.replace('^[a-z\d\-_\s]+$', '', regex = True)
df['CROSS STREET NAME'] = df['CROSS STREET NAME'].str.replace('^[a-z\d\-_\s]+$', '', regex = True)
df['OFF STREET NAME'] = df['OFF STREET NAME'].str.replace('^[a-z\d\-_\s]+$', '', regex = True)

## Missing Data
In the vehicle collision data set, there are some attributes that are missing important information. For example,  with missing value - CONTRIBUTING FACTOR VEHICLE 1-5, we can fill up with ‘UNSPECIFIED’. 


In [16]:
# Fill empty values with 'Unspecified'
df['CONTRIBUTING FACTOR VEHICLE 1'] = df['CONTRIBUTING FACTOR VEHICLE 1'].replace('', 'UNSPECIFIED', regex = True)
df['CONTRIBUTING FACTOR VEHICLE 2'] = df['CONTRIBUTING FACTOR VEHICLE 2'].replace('', 'UNSPECIFIED', regex = True)

As for VEHICLE TYPE CODE 1-2，we can fill up with ‘UNKNOWN’.

In [17]:
df['VEHICLE TYPE CODE 1'] = df['VEHICLE TYPE CODE 1'].replace('', 'UNKNOWN', regex = True)
df['VEHICLE TYPE CODE 2'] = df['VEHICLE TYPE CODE 2'].replace('', 'UNKNOWN', regex = True)

Also from Column 10-17, ‘NUMBER OF PERSONS INJURED’ to NUMBER OF MOTORIST INJURED, replacing the empty value in attributes NUMBER OF PERSONS INJURED/NUMBER OF PERSONS KILLED to 0.

In [18]:
# Replace empty values with '0'
df['NUMBER OF PERSONS INJURED'] = df['NUMBER OF PERSONS INJURED'].str.upper().replace('', '0', regex = True)
df['NUMBER OF PERSONS KILLED'] = df['NUMBER OF PERSONS KILLED'].str.upper().replace('', '0', regex = True)

In [19]:
# Replace empty data
df.loc[df['BOROUGH'] == '', 'BOROUGH'] = 'N/A'
df.loc[df['ZIP CODE'] == '', 'ZIP CODE'] = '00000'
df.loc[df['LATITUDE'] == '', 'LATITUDE'] = '0.0'
df.loc[df['LONGITUDE'] == '', 'LONGITUDE'] = '0.0'
df.loc[df['LOCATION'] == '', 'LOCATION'] = '(0.0, 0.0)'
df.loc[df['ON STREET NAME'] == '', 'ON STREET NAME'] = 'N/A'
df.loc[df['CROSS STREET NAME'] == '', 'CROSS STREET NAME'] = 'N/A'
df.loc[df['OFF STREET NAME'] == '', 'OFF STREET NAME'] = 'N/A'

## Type Error
There are so many type errors, like confusion of case, missing or adding characters and just only errors.

As for CONTRIBUTING FACTOR VEHICLE, some numbers are uselessful, so they will be replaced by “UNSPECIFIED”. 


In [20]:
# CONTRIBUTING FACTOR VEHICLE
df['CONTRIBUTING FACTOR VEHICLE 1'] = df['CONTRIBUTING FACTOR VEHICLE 1'].str.upper().replace('ILLNES', 'ILLNESS', regex = True)
df['CONTRIBUTING FACTOR VEHICLE 1'] = df['CONTRIBUTING FACTOR VEHICLE 1'].str.upper().replace('80', 'UNSPECIFIED', regex = True)
df['CONTRIBUTING FACTOR VEHICLE 1'] = df['CONTRIBUTING FACTOR VEHICLE 1'].str.upper().replace('1', 'UNSPECIFIED', regex = True)
df['CONTRIBUTING FACTOR VEHICLE 2'] = df['CONTRIBUTING FACTOR VEHICLE 2'].str.upper().replace('ILLNES', 'ILLNESS', regex = True)
df['CONTRIBUTING FACTOR VEHICLE 2'] = df['CONTRIBUTING FACTOR VEHICLE 2'].str.upper().replace('80', 'UNSPECIFIED', regex = True)
df['CONTRIBUTING FACTOR VEHICLE 2'] = df['CONTRIBUTING FACTOR VEHICLE 2'].str.upper().replace('1', 'UNSPECIFIED', regex = True)

VEHICLE TYPE CODE is more complex than CONTRIBUTING FACTOR VEHICLE. There are thousands of vehicle types and type errors. For example, just for the ambulance type, there are kinds of spelling, 'AMB', 'AMBU', 'AMBUKANCE', 'AMBUL', 'AMBULACE', 'AMBULANCE', 'AMBULANE', 'AMBULENCE', 'AMBULETTE', 'AMDU', 'AMUBULANCE', 'AMULANCE'. So applying [Standardizing Spellings](https://github.com/VIDA-NYU/openclean/blob/master/examples/notebooks/Standardization%20of%20Ethiopian%20Calendar%20and%20Woreda%20Names.ipynb) is necessary.


In [21]:
# Create a Matcher to match vehicle type
from openclean.function.matching.fuzzy import FuzzySimilarity
from openclean.function.matching.base import DefaultStringMatcher

vehicle_type = set(['SEDAN','4 DR SEDAN','2 DR SEDAN','MOTORCYCL','TAXI',
                    'VAN','TRUCK','BUS','BIKE','MOTORCYCLE',
                    'STATION WAGON / SPORT UTILITY VEHICLE',
                    'LARGE COM VEH','SMALL COM VEH','OTHER',
                    'E-BIKE','E-SCOOTER','AMBULANCE','UNKOWN',
                    'LIVERY VEHICLE','TRACTOR TRUCK DIESEL',
                    'CONVERTIBLE','DUMP','FDNY','USPS','TANK'])

matcher = DefaultStringMatcher(
            vocabulary = vehicle_type,
            similarity = FuzzySimilarity(),
            best_matches_only=True,
            no_match_threshold=0.2,
            cache_results = True)

def standardizeVehicleType(x):
    vtype = ""
    try:
        vtype = matcher.find_matches(x)[0].term
    except TypeError:
        vtype = "UNKNOWN"
    except IndexError:
        vtype = "UNKNOWN"
    return vtype

Applying the function consturcted above to apply to attributes VEHICLE TYPE CODE1 AND VEHICLE TYPE CODE 2

In [22]:
# Apply standardize vehicle type method on VEHICLE TYPE CODE 1 and VEHICLE TYPE CODE 2
df["VEHICLE TYPE CODE 1"] = df["VEHICLE TYPE CODE 1"].map(standardizeVehicleType)
df["VEHICLE TYPE CODE 2"] = df["VEHICLE TYPE CODE 2"].map(standardizeVehicleType)

In [23]:
df

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2
0,04/14/2021,5:32,N/A,00000,0.0,0.0,"(40.80205005, -73.8297471344276)",BRONX WHITESTONE BRG,N/A,N/A,...,0,0,0,0,0,FOLLOWING TOO CLOSELY,UNSPECIFIED,4407480,SEDAN,SEDAN
1,04/13/2021,21:35,BROOKLYN,11217,40.68358,-73.97617,"(40.68358, -73.97617)",N/A,N/A,620 ATLANTIC AVE,...,0,0,0,0,0,UNSPECIFIED,UNSPECIFIED,4407147,SEDAN,UNKOWN
2,04/15/2021,16:15,N/A,00000,0.0,0.0,"(40.849642349999996, -73.83640393750001)",HUTCHINSON RIVER PKWY,N/A,N/A,...,0,0,0,0,0,PAVEMENT SLIPPERY,UNSPECIFIED,4407665,STATION WAGON / SPORT UTILITY VEHICLE,UNKOWN
3,04/13/2021,16:00,BROOKLYN,11222,0.0,0.0,"(42.083058, -76.05075)",VANDERVORT AVE,ANTHONY ST,N/A,...,0,0,0,0,0,FOLLOWING TOO CLOSELY,UNSPECIFIED,4407811,SEDAN,UNKOWN
4,04/12/2021,8:25,N/A,00000,0,0,"(0.0, 0.0)",EDSON AVE,N/A,N/A,...,0,0,0,0,0,UNSPECIFIED,UNSPECIFIED,4406885,STATION WAGON / SPORT UTILITY VEHICLE,SEDAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1841948,07/06/2012,15:09,MANHATTAN,10035,40.8012354,-73.9418153,"(40.8012354, -73.9418153)",EAST 119 ST,PARK AVE,N/A,...,0,0,0,0,0,UNSPECIFIED,UNSPECIFIED,59654,STATION WAGON / SPORT UTILITY VEHICLE,LIVERY VEHICLE
1841949,07/03/2012,17:30,QUEENS,11102,40.7747112,-73.9333863,"(40.7747112, -73.9333863)",27 AVE,4 ST,N/A,...,0,0,0,2,0,FAILURE TO YIELD RIGHT-OF-WAY,UNSPECIFIED,272592,LIVERY VEHICLE,STATION WAGON / SPORT UTILITY VEHICLE
1841950,07/01/2012,15:30,BROOKLYN,11236,40.6450318,-73.9199775,"(40.6450318, -73.9199775)",RALPH AVE,CLARENDON RD,N/A,...,0,0,0,0,0,UNSPECIFIED,UNSPECIFIED,135041,SMALL COM VEH,LIVERY VEHICLE
1841951,07/08/2012,18:30,N/A,00000,40.7861217,-73.8040782,"(40.7861217, -73.8040782)",N/A,N/A,N/A,...,0,0,0,0,0,UNSPECIFIED,UNSPECIFIED,3055617,LIVERY VEHICLE,LIVERY VEHICLE


In [24]:
profile = dataset_profile(df)
profile.stats()

/Users/siweiwang/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname AVE identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


,total,empty,distinct,uniqueness,entropy
CRASH DATE,1811925,0,3426,0.001891,11.679901
CRASH TIME,1811925,0,1440,0.000795,8.926582
BOROUGH,1811925,0,6,0.000003,2.366336
ZIP CODE,1811925,0,233,0.000129,5.950403
LATITUDE,1811925,0,122378,0.067540,14.510685
LONGITUDE,1811925,0,95990,0.052977,14.250261
LOCATION,1811925,185167,243639,0.149770,16.181535
ON STREET NAME,1811925,0,9151,0.005050,9.042799
CROSS STREET NAME,1811925,0,9620,0.005309,7.973008
OFF STREET NAME,1811925,0,167557,0.092475,3.253902


In [25]:
profile.types()

,date,float,int,str,unknown
CRASH DATE,3426,0,0,0,0
CRASH TIME,0,0,0,1440,0
BOROUGH,0,0,0,6,0
ZIP CODE,0,0,232,1,0
LATITUDE,0,122377,1,0,0
LONGITUDE,0,95988,2,0,0
LOCATION,0,0,0,243622,17
ON STREET NAME,0,0,0,9151,0
CROSS STREET NAME,4,0,0,9616,0
OFF STREET NAME,1259,0,0,166298,0


# Output
Export the final result to a new csv file

In [26]:
# Export
compression_opts = dict(method='zip', archive_name='out.csv')  
df.to_csv('out.zip', index=False, compression=compression_opts)